In [21]:
from pyspark.sql.functions import *

In [ ]:
df = spark.read.csv('sales.csv', header=True, inferSchema=True)

In [ ]:
print(df.schema)
df.printSchema()

In [11]:
df.select("Country").distinct().show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



In [10]:
df1.count()

34

In [17]:
df.select("Name","Price").filter("Country=='Brazil'").show()

+-------+-----+-------+
|   Name|Price|country|
+-------+-----+-------+
|Joachim| 1200| Brazil|
|  Diana| 7500| Brazil|
+-------+-----+-------+



In [24]:
df.printSchema()

root
 |-- Transaction_date: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Account_Created: string (nullable = true)
 |-- Last_Login: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [26]:
df.groupBy("Country").agg(sum("Price")).withColumnRenamed('sum(Price)', 'TotalPrice').show()

+-------------+----------+
|      Country|TotalPrice|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



In [46]:
df0 = df.groupBy("Country").agg(sum("Price")).withColumnRenamed('sum(Price)', 'TotalPrice').\
orderBy('TotalPrice', ascending = False)

In [38]:
df2 = spark.read.csv("countries.csv",header=True,inferSchema = True)

In [39]:
df2.printSchema()

root
 |-- Country: string (nullable = true)
 |-- ID: integer (nullable = true)



In [50]:
df.groupBy("Country").agg(sum("Price")).withColumnRenamed('sum(Price)', 'TotalPrice')\
.join(df2,"Country").select("ID","TotalPrice")\
.orderBy('TotalPrice', ascending = False).show()

+---+----------+
| ID|TotalPrice|
+---+----------+
|  2|    350350|
|  1|     63600|
|  8|     42000|
|  5|     30300|
|  7|     29100|
|  3|     22800|
| 19|     22800|
| 12|     19200|
|  6|     14400|
| 16|     12000|
| 33|      8700|
| 15|      8400|
| 13|      8400|
| 26|      3600|
| 22|      3600|
| 28|      3600|
| 14|      3600|
| 10|      3600|
| 24|      2400|
|  9|      2400|
+---+----------+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import *

numOfIterations = 1

lines = spark.read.text("pagerank_data.txt")
# You can also test your program on the follow larger data set:
# lines = spark.read.text("dblp.in")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))
contribs = links.join(ranks,'src').groupBy('src').count().join(links,'src').join(ranks,'src')\
        .select('dst',(col('rank')/col('count')).alias('contri')).show()


+---+------+
|dst|contri|
+---+------+
|  4|   1.0|
|  3|   0.5|
|  2|   0.5|
|  1|   1.0|
|  1|   0.5|
|  3|   0.5|
+---+------+



In [20]:
from pyspark.sql.functions import *

numOfIterations = 10

#lines = spark.read.text("pagerank_data.txt")
# You can also test your program on the follow larger data set:
lines = spark.read.text("dblp.in")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))
for iteration in range(numOfIterations):
# FILL IN THIS PART
    contribs = outdegrees.join(links,'src').join(ranks,'src')\
        .select('dst',(col('rank')/col('count')).alias('contri'))
    ranks = contribs.groupBy('dst').agg(sum("contri").alias('Totalcontri'))\
        .select(col('dst').alias('src'),(0.85*col('Totalcontri')+0.15).alias('rank'))
    
    
ranks.orderBy(desc('rank')).show()

+-----+------------------+
|  src|              rank|
+-----+------------------+
|    3| 16775.73526883919|
|    2|15259.179382798762|
|    1|10159.700782343525|
|    4|  8992.66455229212|
|    5| 7293.343859690132|
|    7| 6000.921547713604|
|    6| 5057.615774722168|
|    8| 2593.763706927988|
|    9|  2394.02981928632|
|   10| 2283.782476601727|
|   13|1862.5063389672428|
|   14| 1822.667679424879|
|14175|1289.2149600703403|
|  117|1095.6979368422024|
|  120| 1088.077815887215|
|   11|1076.9015952513955|
|   16| 994.3828772892236|
|  122| 975.0243284791197|
|   15| 966.2459632448032|
|14173| 939.6263446314646|
+-----+------------------+
only showing top 20 rows



In [19]:
import re
from operator import add

def computeContribs(urls, rank):
    #each destination in urls equally share the page rank
    # Calculates URL contributions to the rank of other URLs.
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)#yield is used in for-loop and writes a new element into the resulting sequence.

def parseNeighbors(urls):
    # Parses a urls pair string into urls pair."""
    parts = urls.split(' ')
    return parts[0], parts[1]

# Loads in input file. It should be in format of:
#     URL(source)         neighbor URL(destination)
#     URL         neighbor URL
#     URL         neighbor URL
#     ...

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/*
#lines = sc.textFile("pagerank_data.txt", 1)
lines = sc.textFile("dblp.in", 5)
#print(lines.take(10))
numOfIterations = 10

# Loads all URLs from input file and initialize their neighbors. 
links = lines.map(lambda urls: parseNeighbors(urls)) \
             .groupByKey()

# Loads all URLs with other URL(s) link to from input file 
# and initialize ranks of them to one.
ranks = links.mapValues(lambda neighbors: 1.0)#do mapping only for value in the list of (key,value) pairs

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(numOfIterations):
    # Calculates URL contributions to the rank of other URLs.
    
    contribs = links.join(ranks)  \
                    .flatMap(lambda url_urls_rank:
                             computeContribs(url_urls_rank[1][0],
                                             url_urls_rank[1][1]))
    # After the join, each element in the RDD is of the form
    # (url, (list of neighbor urls(destinations), rank))
    # join: Return an RDD containing all pairs of elements with matching keys in self and other.
    
    #contribs is of form:[(u1,1/3),(u2,1/3),...,...] key: destincation url, value: contribution to this destination
    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank*0.85+0.15 )
    # ranks = contribs.reduceByKey(add).map(lambda t: (t[0], t[1] * 0.85 + 0.15))

print(ranks.top(5, lambda x: x[1]))#only one action ; for loop only construct the lineage graph



[('3', 16775.735268839184), ('2', 15259.179382798724), ('1', 10159.700782343547), ('4', 8992.664552292099), ('5', 7293.343859690148)]
